In [16]:
model_path = "hf-internal-testing/Mixtral-tiny"
token_path = "mistralai/Mixtral-8x7B-v0.1"

import numpy as np
from transformers import AutoTokenizer, MixtralModel, MixtralConfig, FlaxMixtralModel
config = MixtralConfig.from_pretrained(model_path)
config.output_router_logits = True
config.output_attentions = True
config.output_hidden_states = True
print(config.max_position_embeddings)
config.max_position_embeddings = 4096
# print(config.max_position_embeddings)

131072


In [12]:
model = MixtralModel.from_pretrained(model_path, config=config)
tokenizer = AutoTokenizer.from_pretrained(token_path)

prompt = "Hey, are you conscious? Can you talk to me?"
inputs = tokenizer(prompt, return_tensors="pt")

x = model(**inputs)


In [13]:
print(model)

MixtralModel(
  (embed_tokens): Embedding(32000, 1024)
  (layers): ModuleList(
    (0-1): 2 x MixtralDecoderLayer(
      (self_attn): MixtralSdpaAttention(
        (q_proj): Linear(in_features=1024, out_features=1024, bias=False)
        (k_proj): Linear(in_features=1024, out_features=256, bias=False)
        (v_proj): Linear(in_features=1024, out_features=256, bias=False)
        (o_proj): Linear(in_features=1024, out_features=1024, bias=False)
        (rotary_emb): MixtralRotaryEmbedding()
      )
      (block_sparse_moe): MixtralSparseMoeBlock(
        (gate): Linear(in_features=1024, out_features=8, bias=False)
        (experts): ModuleList(
          (0-7): 8 x MixtralBlockSparseTop2MLP(
            (w1): Linear(in_features=1024, out_features=3584, bias=False)
            (w2): Linear(in_features=3584, out_features=1024, bias=False)
            (w3): Linear(in_features=1024, out_features=3584, bias=False)
            (act_fn): SiLU()
          )
        )
      )
      (input_laye

In [3]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

count_parameters(model)

214193152

In [7]:
x.last_hidden_state

tensor([[[-1.4910,  1.2440,  2.3478,  ...,  1.4659, -1.9996,  2.3321],
         [-1.4066, -1.2585, -0.1159,  ...,  0.4500,  0.3790,  0.8595],
         [-2.1107,  0.0814, -0.1963,  ..., -0.8745, -0.6745, -0.0833],
         ...,
         [ 0.2947, -1.0839,  0.3771,  ..., -0.5531, -1.0748,  0.6421],
         [-0.1680, -0.6061,  1.2885,  ..., -1.1196,  1.1017, -0.3771],
         [ 0.7433,  0.3478,  0.2059,  ...,  1.2677,  0.0713, -0.5899]]],
       grad_fn=<MulBackward0>)

In [17]:
model = FlaxMixtralModel.from_pretrained(model_path, config=config)
tokenizer = AutoTokenizer.from_pretrained(token_path)

prompt = "Hey, are you conscious? Can you talk to me?"
inputs = tokenizer(prompt, return_tensors="jax")

# y = model(**inputs)

Some weights of the model checkpoint at hf-internal-testing/Mixtral-tiny were not used when initializing FlaxMixtralModel: {('lm_head', 'kernel')}
- This IS expected if you are initializing FlaxMixtralModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing FlaxMixtralModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some of the weights of FlaxMixtralModel were initialized in float16 precision from the model checkpoint at hf-internal-testing/Mixtral-tiny:
[('embed_tokens', 'embedding'), ('layers', '0', 'block_sparse_moe', 'experts', '0', 'w1', 'kernel'), ('layers', '0', 'block_sparse_moe', 'experts', '0', 'w2', 'kernel'), ('layers', '0', 'block_sparse_moe', 'experts', '0', 'w3', 'kernel'), ('laye

In [22]:
print(model._params_shape_tree)

FrozenDict({
    embed_tokens: {
        embedding: ShapeDtypeStruct(shape=(32000, 1024), dtype=float32),
    },
    layers: {
        0: {
            block_sparse_moe: {
                experts: {
                    0: {
                        w1: {
                            kernel: ShapeDtypeStruct(shape=(1024, 3584), dtype=float32),
                        },
                        w2: {
                            kernel: ShapeDtypeStruct(shape=(3584, 1024), dtype=float32),
                        },
                        w3: {
                            kernel: ShapeDtypeStruct(shape=(1024, 3584), dtype=float32),
                        },
                    },
                    1: {
                        w1: {
                            kernel: ShapeDtypeStruct(shape=(1024, 3584), dtype=float32),
                        },
                        w2: {
                            kernel: ShapeDtypeStruct(shape=(3584, 1024), dtype=float32),
                        }

In [4]:
y.last_hidden_state

Array([[[-1.4910138 ,  1.2440001 ,  2.347837  , ...,  1.4658867 ,
         -1.9995558 ,  2.3321059 ],
        [-1.4065796 , -1.2585348 , -0.11586622, ...,  0.4500268 ,
          0.37901428,  0.8594837 ],
        [-2.1106687 ,  0.08143321, -0.19634205, ..., -0.87450725,
         -0.67447233, -0.08327945],
        ...,
        [ 0.2947395 , -1.0838611 ,  0.377108  , ..., -0.55314595,
         -1.074801  ,  0.6420861 ],
        [-0.16801266, -0.60607636,  1.2884645 , ..., -1.1196154 ,
          1.1017209 , -0.37705478],
        [ 0.7433374 ,  0.3478445 ,  0.20590468, ...,  1.2677363 ,
          0.07131021, -0.5898897 ]]], dtype=float32)

In [ ]:
to_keep = []
for i in range(len(x.attentions)):
        for j in range(len(x.attentions[i])):
                for k in range(len(x.attentions[i][j])):
                        for l in range(len(x.attentions[i][j][k])):
                                to_keep.append(np.amax(np.abs(np.array(x.attentions[i][j][k][l].detach().cpu()) - np.array(y.attentions[i][j][k][l]))))

# for i in range(len(x.last_hidden_state)):
#         for j in range(len(x.last_hidden_state[i])):
#                                 to_keep.append(np.amax(np.abs(np.array(x.attentions[i][j].detach().cpu()) - np.array(y.attentions[i][j]))))

to_keep.append(np.amax(np.abs(np.array(x.attentions[0].detach().cpu()) - np.array(y.attentions[0]))))


for i in range(len(x.hidden_states)):
        for j in range(len(x.hidden_states[i])):
                for k in range(len(x.hidden_states[i][j])):
                                to_keep.append(np.amax(np.abs(np.array(x.hidden_states[i][j][k].detach().cpu()) - np.array(y.hidden_states[i][j][k]))))


In [ ]:
to_keep = []
for i in range(len(x.attentions)):
        for j in range(len(x.attentions[i])):
                for k in range(len(x.attentions[i][j])):
                        for l in range(len(x.attentions[i][j][k])):
                                to_keep.append(np.amax(np.abs(np.array(x.attentions[i][j][k][l].detach().cpu()) - np.around(np.array(y.attentions[i][j][k][l]), 4))))

# for i in range(len(x.last_hidden_state)):
#         for j in range(len(x.last_hidden_state[i])):
#                                 to_keep.append(np.amax(np.abs(np.array(x.attentions[i][j].detach().cpu()) - np.array(y.attentions[i][j]))))

to_keep.append(np.amax(np.abs(np.array(x.attentions[0].detach().cpu()) - np.around(np.array(y.attentions[0]), 4))))


for i in range(len(x.hidden_states)):
        for j in range(len(x.hidden_states[i])):
                for k in range(len(x.hidden_states[i][j])):
                                to_keep.append(np.amax(np.abs(np.array(x.hidden_states[i][j][k].detach().cpu()) - np.around(np.array(y.hidden_states[i][j][k]), 4))))
